# TiVo - ETL Pipeline - Preparing Data for Clustering

**NOTE**: All cells that have Excercises in them are marked with `#EXERCISE-TODO-[NO OF EXERCISE]`

## Import General Packages

In [ ]:
%pip install -r requirements.txt

In [1]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore

## E: Extract and explore the data  (EXTRACT part of ETL pipeline)

### Read in from Excel file

In [2]:
dtivo_master = pd.read_excel('./data/tivo_subset.xlsx',
                             index_col=0)            #Read in the first column (ID) as the index
dtivo_master.head()

,Gender,Education,Income,Age,PurchasePoint,ElecSpend_Annual,Tvhours,TechAdopt,Fav_Feature
ID,,,,,,,,,
1,male,none,49,30,mass-consumer electronics,420,2,late,saving favorite shows to watch as a family
2,male,none,46,36,mass-consumer electronics,420,10,late,saving favorite shows to watch as a family
3,male,BA,58,66,specialty stores,768,0,early,time shifting
4,male,PhD,51,78,mass-consumer electronics,396,5,late,saving favorite shows to watch as a family
5,female,none,46,52,mass-consumer electronics,540,2,late,saving favorite shows to watch as a family


### Get a feel of the data

In [3]:
dtivo_master.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 1 to 1000
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Gender            1000 non-null   object
 1   Education         1000 non-null   object
 2   Income            1000 non-null   int64 
 3   Age               1000 non-null   int64 
 4   PurchasePoint     1000 non-null   object
 5   ElecSpend_Annual  1000 non-null   int64 
 6   Tvhours           1000 non-null   int64 
 7   TechAdopt         1000 non-null   object
 8   Fav_Feature       1000 non-null   object
dtypes: int64(4), object(5)
memory usage: 78.1+ KB


### Exploratory Analyses: explore possible bases for segmentation

In [4]:
dtivo_master.groupby('TechAdopt')['Fav_Feature'].value_counts()

TechAdopt  Fav_Feature                               
early      cool gadget                                   228
           schedule control                              221
           time shifting                                 221
           programming/interactive features              130
late       saving favorite shows to watch as a family    200
Name: count, dtype: int64

*Q: Did we learn anything about the product feature that is of interest to early vs. late adopters?  Which ones will you market to the premium market segment?*

Ans:

In [ ]:
#EXERCISE-TODO-01: Obtain grouped value counts for Purchase_Point
#and write in commentary under it, what ordinal scale would
#suitable?
dtivo_master.groupby('TechAdopt')[__________________].value_counts()


## T: Data Transformations  (TRANSFORM part of ETL pipeline)

In [6]:
#Make a copy of the master DataFrame to apply
#transformations to.  We may need the original master file
#so this is a good practice to apply transformations to a copy
dtivo = dtivo_master.copy()

### Factorizations

#### Fav_Feature

In [18]:
#Map the zipped dictionary from 'Fav_Feature' to 'Fav_Coded'
dtivo['Fav_Coded'], fav_uniques = dtivo['Fav_Feature'].factorize()
print(fav_uniques)


Index(['saving favorite shows to watch as a family', 'time shifting',
       'cool gadget', 'schedule control', 'programming/interactive features'],
      dtype='object')


In [19]:
dtivo.head(10)

,Gender,Education,Income,Age,PurchasePoint,ElecSpend_Annual,Tvhours,TechAdopt,Fav_Feature,Fav_Coded
ID,,,,,,,,,,
1,male,none,49,30,mass-consumer electronics,420,2,late,saving favorite shows to watch as a family,0
2,male,none,46,36,mass-consumer electronics,420,10,late,saving favorite shows to watch as a family,0
3,male,BA,58,66,specialty stores,768,0,early,time shifting,1
4,male,PhD,51,78,mass-consumer electronics,396,5,late,saving favorite shows to watch as a family,0
5,female,none,46,52,mass-consumer electronics,540,2,late,saving favorite shows to watch as a family,0
6,female,BA,31,72,retail,168,1,early,time shifting,1
7,male,none,33,62,discount,216,0,early,cool gadget,2
8,male,none,29,30,retail,276,1,early,schedule control,3
9,male,none,57,60,specialty stores,888,0,early,schedule control,3


#### Education
Sometimes the ordering of the numbers will matter, for example, in Education, we want to preserve our own order - especially when we see the results that summarize clusters.  So we "Zip" the categories with our own factors.

In [7]:
ed_types = dtivo['Education'].unique()
print(ed_types)

['none' 'BA' 'PhD' 'MA']


In [ ]:
dtivo['Education'].factorize(sort=True)

In [8]:
ed_levels = [1,2,4,3]  #order numbers matched against ed_types list
dict_edtypes = dict(zip(ed_types,ed_levels))
print(dict_edtypes)

{'none': 1, 'BA': 2, 'PhD': 4, 'MA': 3}


In [ ]:
dtivo['Ed_Coded'] = dtivo['Education'].map(dict_edtypes)
dtivo.head(10)

#### PurchasePoint 

In [21]:
#EXERCISE-TODO-02: Decide whether you would like to 
#code 'PurchasePoint' using simply factorization (as we showed in Favorite Feature) 
# OR by mapping your own 'zipped' dictionary.
#HINT: CONSIDER YOUR FINDINGS FROM TODO-01
# WHICHEVER WAY YOU, CHOSE SAVE THE RESULT


#?
#?
#?
dtivo["PPoint_Coded"] = dtivo['Education']._____________________________
dtivo.head(10)


#### Factorizing a binary - what to label the result?

In [23]:
dtivo['Gender'].head()

ID
1      male
2      male
3      male
4      male
5    female
Name: Gender, dtype: object

In [34]:
dtivo['Gender'].factorize(sort=True)

(array([1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1,
        0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0,
        0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
        0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0,
        1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
        1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
        0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,
        0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
        1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 

When factorizing a binary variable, you need to anticipate which category gets zero and which gets one.  Factorize offers to first sort the 'unique' categories.  For example, our dataset only has two genders (male and female) so this is a binary variable.  If we apply sort, we can anticipate that as 'F' comes before 'M', females will get `0` and males will get assigned `1`.  In this case the Series we get is Male as males have `1`.

 Without this, what ever the first category appearing in the first row is will get 0 and the next category will get 1 (so given the above data, male would get `0` without sort)

In [35]:
dtivo['Male'], gender_uniques = dtivo['Gender'].factorize(sort=True)
print(gender_uniques)
dtivo.head()

Index(['female', 'male'], dtype='object')


,Gender,Education,Income,Age,PurchasePoint,ElecSpend_Annual,Tvhours,TechAdopt,Fav_Feature,Fav_Coded,Female,Male
ID,,,,,,,,,,,,
1,male,none,49,30,mass-consumer electronics,420,2,late,saving favorite shows to watch as a family,0,0,1
2,male,none,46,36,mass-consumer electronics,420,10,late,saving favorite shows to watch as a family,0,0,1
3,male,BA,58,66,specialty stores,768,0,early,time shifting,1,0,1
4,male,PhD,51,78,mass-consumer electronics,396,5,late,saving favorite shows to watch as a family,0,0,1
5,female,none,46,52,mass-consumer electronics,540,2,late,saving favorite shows to watch as a family,0,1,0


### Rows to keep for clustering
We are only interested at launch in segments *within* early adopters => filter out late adopters.


In [38]:
#Assuming we have done some segment targeting already - like focus on Early adopters only.
dtivo_early = dtivo[dtivo['TechAdopt']=='early'].copy()
len(dtivo_early)

800

In [39]:
outlierIDs = []   #WE CAN DROP ANY CASES THAT WE THINK ARE OUTLIERS AND WILL DISTORT OUR SEGMENTS
#EXERCISE-TODO-Nth: RETURN TO THIS AFTER RUNNING THE CLUSTER ANALYSIS
dtivo_early.drop(index=outlierIDs, inplace=True)

In [40]:
len(dtivo_early)

800

### Columns to keep for clustering

In [41]:
colsAll = list(dtivo_early.columns)
colsAll

['Gender',
 'Education',
 'Income',
 'Age',
 'PurchasePoint',
 'ElecSpend_Annual',
 'Tvhours',
 'TechAdopt',
 'Fav_Feature',
 'Fav_Coded',
 'Female',
 'Male']

The word 'descriptors' signifies that these are the chosen variables that we believe will truly help us predict our segmentation.  Descriptors chosen should be those that we will know about the customer before the marketing action takes place.  For example, before selling our product, will we know their purchase point?

In [44]:

colsToDrop = ['Education',      #factorized, only keeping its coded form
              'PurchasePoint',  #factorized, only keeping its coded form
              'Tvhours',        #Dropping to keep it simple
              'TechAdopt',      #Don't need after filtering out TechAdopter==late.
              'Fav_Feature',    #factorized, only keeping coded form
              'Gender',         #already have 'Male' after factorization
              'Fav_Coded',      #Dropping to keep it simple
#              'PPoint_Coded',   #Dropping to keep it simple in first run - uncomment after TODOS
                                #EXERCISE-TODO-TAKEHOME - try doing this again but with some of the variables we dropped for simplicity
             ]

dtivo_descriptors = dtivo_early.drop(columns=colsToDrop)
colsKept = list(dtivo_descriptors.columns)
print(colsKept)
dtivo_descriptors.head()

['Income', 'Age', 'ElecSpend_Annual', 'Male']


,Income,Age,ElecSpend_Annual,Male
ID,,,,
3,58,66,768,1
6,31,72,168,0
7,33,62,216,1
8,29,30,276,1
9,57,60,888,1


### Normalize using Standardization

If Clustering has to give equal weight to each field (or 'feature' in ML terms), then<br>
we must make sure they are on the same scale (i.e. **Normalize** the fields).

If we are interested in how our segments vary from the average (e.g. <br>
above average income, highly below average age, etc.), then we use<br>
**Standardization** for normalization.


In [45]:
#IMPORT PACKAGE FOR STANDARDIZATION
from sklearn import preprocessing

In [46]:
#STANDARDIZED SCALING
dtivo_std = pd.DataFrame(preprocessing.scale(dtivo_descriptors),
                         columns=colsKept,                 #TODO: REVIEW in light of any changes to COLUMNS DROPPED
                         index=dtivo_descriptors.index
                        )

#only for checking if done
dtivo_std.describe().round(2)

,Income,Age,ElecSpend_Annual,Male
count,800.00,800.00,800.00,800.00
mean,0.00,-0.00,0.00,-0.00
std,1.00,1.00,1.00,1.00
min,-0.46,-1.59,-1.19,-1.11
25%,-0.26,-0.94,-0.69,-1.11
50%,-0.17,-0.00,-0.52,0.90
75%,-0.05,0.87,0.37,0.90
max,20.26,1.81,3.33,0.90


In [47]:
dtivo_std.head()

,Income,Age,ElecSpend_Annual,Male
ID,,,,
3,0.621504,1.038894,1.989377,0.902252
6,-0.167615,1.367897,-0.800871,-1.108338
7,-0.109161,0.819559,-0.577651,0.902252
8,-0.226068,-0.935121,-0.298626,0.902252
9,0.592277,0.709892,2.547427,0.902252


## L: Loading the transformed data (LOAD part of ETL pipeline)

In [50]:
dtivo_descriptors.to_csv("data/dtivo_descriptors.csv")

In [51]:
dtivo_std.to_csv("data/dtivo_standardized.csv")